In [1]:
import pandas as pd
import joblib
import numpy as np

try:
    model = joblib.load('linear_regression_energy_model.pkl')
    model_features = joblib.load('linear_model_features.pkl')
    scaler = joblib.load('scaler.pkl') # Load the trained scaler
    print("Model, features, and scaler loaded successfully.")
    print(f"Model expects features: {model_features}")
except FileNotFoundError as e:
    print(f"Error loading files: {e}. Please ensure 'linear_regression_energy_model.pkl', 'linear_model_features.pkl', and 'scaler.pkl' are in the same directory.")
    print("Exiting. Please run the training script first to generate these files.")
    exit() 

Model, features, and scaler loaded successfully.
Model expects features: ['square_footage', 'number_of_occupants', 'appliances_used', 'building_type_Industrial']


In [2]:
def prepare_new_input(input_dict, model_features, scaler):
    input_df = pd.DataFrame([input_dict])
    processed_input = pd.DataFrame(0, index=[0], columns=model_features)

    numerical_cols = ['square_footage', 'number_of_occupants', 'appliances_used', 'average_temperature']
    for col in numerical_cols:
        if col in input_df.columns:
            processed_input[col] = input_df[col]

    if 'building_type' in input_df.columns:
        b_type = input_df['building_type'].iloc[0]
        col_name = f'building_type_{b_type}'
        if col_name in processed_input.columns:
            processed_input[col_name] = 1

    if 'day_of_week' in input_df.columns:
        d_week = input_df['day_of_week'].iloc[0]
        col_name = f'day_of_week_{d_week}'
        if col_name in processed_input.columns:
            processed_input[col_name] = 1

    final_input_df = processed_input[model_features]
    final_input_array = final_input_df.values
    scaled_input = scaler.transform(final_input_array)

    return scaled_input

In [3]:
new_building_data_1 = {
    'square_footage': 7063,
    'number_of_occupants': 76,
    'appliances_used': 10,
    'average_temperature': 29.84,
    'building_type': 'Residential'
}

new_building_data_2 = {
    'square_footage': 9578,
    'number_of_occupants': 54,
    'appliances_used': 47,
    'average_temperature': 13.56,
    'building_type': 'Commercial'
}

new_building_data_3 = {
    'square_footage': 20920,
    'number_of_occupants': 89,
    'appliances_used': 31,
    'average_temperature': 32.76,
    'building_type': 'Industrial',

}

new_building_data_4 = {
    'square_footage': 3000,
    'number_of_occupants': 2,
    'appliances_used': 20,
    'average_temperature': 15.67,
    'building_type': 'Commercial'
    
}

print("\n--- Making Predictions ---")
prepared_input_1 = prepare_new_input(new_building_data_1, model_features, scaler)
predicted_energy_1 = model.predict(prepared_input_1)
print(f"Predicted Energy Consumption for Input 1 (Residential, Weekday): {predicted_energy_1[0]:.2f}")

prepared_input_2 = prepare_new_input(new_building_data_2, model_features, scaler)
predicted_energy_2 = model.predict(prepared_input_2)
print(f"Predicted Energy Consumption for Input 2 (Commercial, Weekday): {predicted_energy_2[0]:.2f}")

prepared_input_3 = prepare_new_input(new_building_data_3, model_features, scaler)
predicted_energy_3 = model.predict(prepared_input_3)
print(f"Predicted Energy Consumption for Input 3 (Industrial, Weekend): {predicted_energy_3[0]:.2f}")

prepared_input_4 = prepare_new_input(new_building_data_4, model_features, scaler)
predicted_energy_4 = model.predict(prepared_input_4)
print(f"Predicted Energy Consumption for Input 4 (Industrial, Weekend): {predicted_energy_4[0]:.2f}")


--- Making Predictions ---
Predicted Energy Consumption for Input 1 (Residential, Weekday): 2953.40
Predicted Energy Consumption for Input 2 (Commercial, Weekday): 3635.80
Predicted Energy Consumption for Input 3 (Industrial, Weekend): 4970.43
Predicted Energy Consumption for Input 4 (Industrial, Weekend): 2226.98
